In [17]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Настройка Selenium для работы с Chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Функция для парсинга вакансий с HH.ru
def parse_vacancies(role, region):
    url = f"https://hh.ru/search/vacancy?text={role}&area={region}&page=0"
    driver.get(url)

    # Увеличение времени ожидания
    WebDriverWait(driver, 15).until(
        EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'div.vacancy-serp-item'))
    )

    vacancies = []

    for page in range(1):  # Замените на большее количество страниц по необходимости
        print(f"Парсинг страницы {page + 1}...")
        
        job_elements = driver.find_elements(By.CSS_SELECTOR, 'div.vacancy-serp-item')
        print(f"Найдено {len(job_elements)} вакансий на странице")

        for job in job_elements:
            try:
                title_element = job.find_element(By.CSS_SELECTOR, 'a.serp-item__title')
                title = title_element.text
                link = title_element.get_attribute('href')

                company_element = job.find_element(By.CSS_SELECTOR, 'a.vacancy-serp-item__meta-info-company')
                company = company_element.text

                location_element = job.find_element(By.CSS_SELECTOR, 'div.vacancy-serp-item__info span')
                location = location_element.text

                vacancies.append({
                    'Название': title,
                    'Компания': company,
                    'Локация': location,
                    'Ссылка': link
                })
            except Exception as e:
                print(f"Ошибка при парсинге вакансии: {e}")
                continue

        # Переход на следующую страницу
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'a.HH-Pager-Controls-Next')
            next_button.click()
            WebDriverWait(driver, 15).until(
                EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'div.vacancy-serp-item'))
            )
        except Exception as e:
            print("Следующая страница не найдена")
            break

    return vacancies

# Определяем направления и регионы
roles = ['Data Analyst', 'Data Scientist', 'Data Engineer']
regions = {
    'Москва': 1,
    'Санкт-Петербург': 2,
    'Нижний Новгород': 66,
    'Казань': 88
}

# Парсим вакансии по всем направлениям и регионам
all_vacancies = []

for role in roles:
    for region_name, region_code in regions.items():
        print(f"Парсинг вакансий по роли {role} в регионе {region_name}...")
        vacancies = parse_vacancies(role, region_code)
        for vacancy in vacancies:
            vacancy['Направление'] = role
            vacancy['Регион'] = region_name
        all_vacancies.extend(vacancies)

# Преобразуем данные в DataFrame и сохраняем в файл
df = pd.DataFrame(all_vacancies)
df.to_csv('vacancies_data.csv', index=False)
print("Парсинг завершен")
df.head()

# Закрываем браузер
driver.quit()


Парсинг вакансий по роли Data Analyst в регионе Москва...


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00925523+24195]
	(No symbol) [0x008BAA04]
	(No symbol) [0x007B2093]
	(No symbol) [0x007F6ED2]
	(No symbol) [0x007F711B]
	(No symbol) [0x008376F2]
	(No symbol) [0x0081AB84]
	(No symbol) [0x00835280]
	(No symbol) [0x0081A8D6]
	(No symbol) [0x007EBA27]
	(No symbol) [0x007EC43D]
	GetHandleVerifier [0x00BECE13+2938739]
	GetHandleVerifier [0x00C3EC69+3274185]
	GetHandleVerifier [0x009B09C2+594722]
	GetHandleVerifier [0x009B7EDC+624700]
	(No symbol) [0x008C37CD]
	(No symbol) [0x008C0528]
	(No symbol) [0x008C06C5]
	(No symbol) [0x008B2CA6]
	BaseThreadInitThunk [0x76C7FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x776B809E+286]
	RtlGetAppContainerNamedObjectPath [0x776B806E+238]
